## Determination of relationship between the molecular weight and gyration radius of proteins

It is useful to be able to estimate the radius of a particle based on its molecular weight, as the radius of a particle can be used to calculate an estimated diffusion coefficient of said particle. To that end, this notebook was used to find an equation accurately describing the relationship between the molecular weight of a protein and its gyration radius.

Data from Tyn, M.T. and Gusek, T.W. (1990), Prediction of diffusion coefficients of proteins. Biotechnol. Bioeng., 35: 327-338. https://doi.org/10.1002/bit.260350402

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
data = pd.read_excel('protein-mass_vs_gyration-radius.xlsx')
data

,Da,"Radius of gyration, Å"
0,12640,14.8
1,13700,14.8
2,17000,14.8
3,13300,14.5
4,14180,16.7
...,...,...
193,39000000,924.0
194,40000000,924.0
195,40590000,924.0
196,40700000,924.0
